In [2]:
import numpy as np
import json
from math import ceil

In [9]:
f = open('players-2023.json', 'r')
players = json.load(f)
players2 = {}

NAME = "NAME"
ESPN_AVG = "AVG_ESPN"
YAHOO_AVG = "Y!_ AVG"
VALUED_AT = "VALUED AT"
RANK = "R#"
POINTS = "TOTAL_ESPN"
SCALE = 1

for player in players:
    player[YAHOO_AVG] = int(float(player[YAHOO_AVG][1:]))
    player[ESPN_AVG] = int(float(player[ESPN_AVG][1:]))
    player[VALUED_AT] = int(float(player[VALUED_AT][1:]))
    players2[player[NAME]] = player
players[0]

{'R': 1,
 'NAME': 'Nikola Jokic',
 'POS': 'C',
 'TEAM': 'DEN',
 'VALUED AT': 123,
 'Y!_ AVG': 74,
 'AVG_ESPN': 71,
 'TOTAL_ESPN': 4158,
 'PTS': 1690,
 'TREB': 817,
 'AST': 678,
 'STL': 87,
 'BLK': 47,
 '3PM': 57,
 'FGM': 646,
 'FGA': 1022,
 'FTM': 341,
 'FTA': 415,
 'TO': 247,
 'IMG': 'https://www.basketball-reference.com/req/202106291/images/headshots/jokicni01.jpg',
 'IMG2': 'Picture'}

In [4]:
def get_roster(names, weights, W, K):
    dp = []
    pl = []
    for i in range(len(names)):
        dp.append([])
        pl.append([])
        for j in range(W+1):
            dp[i].append([])
            pl[i].append([])
            for k in range(K+1):
                dp[i][j].append(0)
                pl[i][j].append([])
    for i in range(1, len(names)):
        for j in range(1, W+1):
            for k in range(1, K+1):
                if (j >= weights[i-1]):
                    if dp[i-1][j][k] > dp[i-1][j-weights[i-1]][k-1] + names[i-1]:
                        dp[i][j][k] = dp[i-1][j][k]
                        pl[i][j][k] += pl[i-1][j][k]
                    else:
                        dp[i][j][k] = dp[i-1][j-weights[i-1]][k-1] + names[i-1]
                        pl[i][j][k] += pl[i-1][j-weights[i-1]][k-1] + [i-1]
                else:
                    dp[i][j][k] = dp[i-1][j][k]
                    pl[i][j][k] += pl[i-1][j][k]
    return dp[len(names)-1][W][K], pl[len(names)-1][-1][-1]

In [52]:
def get_final_team(scale=ESPN_AVG, salary=200, num_players=13):
    points = [player[POINTS] for player in players]
    weights = [player[AVG_SCALE] for player in players]
    total_points, player_index = get_roster(points, weights, TOTAL_SALARY, NUM_PLAYERS)
    names = []
    print(f'POINTS: {total_points}, AVG_POINTS: {total_points / NUM_PLAYERS}')
    for i in player_index:
        print(f'${players[i][AVG_SCALE]} {players[i][NAME]}')
        names.append((players[i][NAME], players[i][AVG_SCALE]))
    return names

In [63]:
def get_optimal_team(scale=ESPN_AVG, salary=200, num_players=13):
    points = [player[POINTS] for player in players]
    weights = [player[AVG_SCALE] for player in players]
    total_points, player_index = get_roster(points, weights, TOTAL_SALARY, NUM_PLAYERS)
    names = []
    for i in player_index:
        names.append((players[i][NAME], players[i][AVG_SCALE]))
    return names

In [64]:
def auction_player(name, price):
    for player in players:
        if player[NAME] == name:
            player[ESPN_AVG] = ceil(price)

In [65]:
def get_price(name):
    for player in players:
        if player[NAME] == name:
            return player[ESPN_AVG]

In [66]:
def remove_player(name):
    auction_player(name, 99999999999)

In [90]:
def should_draft(name, starting_price, salary, num_players, inc=5, scale=ESPN_AVG):
    print(f'=== SEARCHING MAX PRICE FOR {name} ===')
    og_price = get_price(name)
    init_price = max(starting_price, og_price)
    for i in range(init_price, 200, inc):
        auction_player(name, i)
        players = get_optimal_team(scale=scale, salary=salary, num_players=num_players)
        if (name, i) in players:
            print(f'DRAFT: {name} @ {i}')
        else:
            auction_player(name, og_price)
            break
    print(f'=== DONE SEARCHING MAX PRICE FOR {name} ===')

# THE DRAFT

## DESIRED TEAM

In [91]:
TOTAL_SALARY = 200
NUM_PLAYERS = 12
AVG_SCALE = ESPN_AVG
desired_team = get_final_team(AVG_SCALE, TOTAL_SALARY, NUM_PLAYERS)

POINTS: 36525, AVG_POINTS: 3043.75
$71 Nikola Jokic
$10 Kristaps Porzingis
$10 Fred VanVleet
$12 Zach LaVine
$28 De'Aaron Fox
$9 Evan Mobley
$20 Nikola Vucevic
$7 Nicolas Claxton
$5 Ja Morant
$2 Brook Lopez
$16 Julius Randle
$6 CJ McCollum


### LIVE DRAFT

In [94]:
should_draft(name='Nikola Jokic', 
             starting_price=71,
             num_players=NUM_PLAYERS,
             inc=1,
             salary=TOTAL_SALARY)

=== SEARCHING MAX PRICE FOR Nikola Jokic ===
DRAFT: Nikola Jokic @ 71
DRAFT: Nikola Jokic @ 72
DRAFT: Nikola Jokic @ 73
DRAFT: Nikola Jokic @ 74
DRAFT: Nikola Jokic @ 75
=== DONE SEARCHING MAX PRICE FOR Nikola Jokic ===
